In [ ]:
import numpy as np
import time
import matplotlib.pyplot as plt

In [ ]:
# Benchmark quadratische Matrizenmultiplikationen (np.randn)  -> einmal mit for-loop (Spalte * Zeile)
#                                                             -> dann mit @ (numpy)
# -> Variable Größe als Input
# -> Plotten (Vergleich)

In [ ]:
def matmul(size: int) -> float:
    x = np.random.randn(size, size)
    y = np.random.randn(size, size)
    res = np.zeros((size, size))

    loop_start_time = time.time()
    for i in range(size):
        for j in range(size):
            for k in range(size):
                res[i][j] += x[i][k] * y[k][j]
    loop_end_time = time.time()
    loop_time = loop_end_time - loop_start_time

    numpy_start_time = time.time()
    res2 = x @ y
    numpy_end_time = time.time()
    numpy_time = numpy_end_time - numpy_start_time

    return size, loop_time, numpy_time

Plot Time Differences

In [ ]:
sz, lt, nt = matmul(1000)

plt.bar(sz, lt)
plt.bar(sz, nt)
plt.xticks(sz)

plt.show()

In [ ]:
sizes = []
l_times = []
n_times = []

for i in range(100):
    sz, lt, nt = matmul(i)
    sizes.append(sz)
    l_times.append(lt)
    n_times.append(nt)

plt.bar(sizes, l_times)
plt.bar(sizes, n_times)

plt.legend(["Loop Times", "Numpy Times"])
plt.xticks(sizes)

plt.show()